In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import os 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader

from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *

from facenet_pytorch import InceptionResnetV1
from GAN_model import Generator, Discriminator
from utils.loss import*

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SRGAN parameters

batch_size = 32
num_epochs = 30
lr = 1e-4
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/home/jupyter/dataset/LR_56/train/'
img_path_ref = '/home/jupyter/dataset/HR/train/'

#img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/train/'
#img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
netG = Generator(upscale_factor).to(device)
netD = Discriminator().to(device)

#netG.weight_init(mean=0.0, std=0.2)
#netD.weight_init(mean=0.0, std=0.2)

feature_extraction_model = InceptionResnetV1(pretrained='vggface2').eval()

generator_criterion = GeneratorLoss().to(device)
face_loss = FaceIdentityLoss(feature_extraction_model).to(device)
MSE_criterion = nn.MSELoss()

In [5]:
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
    
results = {'d_loss': [], 'g_loss': [], 'd_score': [],
           'g_score': [], 'psnr': [], 'ssim': []}

In [6]:
# Let's pretrain the Generator
pre_train_epochs = 5
for epoch in range(1, pre_train_epochs+1):
    
    netG.train()
    
    for data, target in training_data_loader:
        data, target = data.to(device), target.to(device)
        
        netG.zero_grad()
        loss = MSE_criterion(netG(data), target)
        loss.backward()
        optimizerG.step()
    
    print('Last loss: {}'.format(loss.item()))
    print('Finished pre-training {}/{}'.format(epoch,pre_train_epochs))

Last loss: 0.0032183481380343437
Finished pre-training 1/5
Last loss: 0.002669692039489746
Finished pre-training 2/5
Last loss: 0.0021901081781834364
Finished pre-training 3/5
Last loss: 0.0023981535341590643
Finished pre-training 4/5
Last loss: 0.002246673684567213
Finished pre-training 5/5


In [7]:
out_path = 'results/'
out_model_path = 'models/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)  

In [8]:
def train(epoch):
    
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 
                       'd_score': 0, 'g_score': 0}
    
    netG.train()
    netD.train()
    iteration = 0
    for data, target in training_data_loader:
        iteration+=1
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target).to(device)
        z = Variable(data).to(device)
        
        fake_img = netG(z)

        netD.zero_grad()
        
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        
        d_loss = 1 - real_out + fake_out
        
        if d_loss.item() > 0.4:
            d_loss.backward(retain_graph=True)
            optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss + FaceID Loss
        ###########################
        netG.zero_grad()
        
        #mse_adversarial_content_loss = generator_criterion(fake_out, fake_img, real_img)
        #face_perceptual_loss = face_loss(fake_img, real_img)
        
        g_loss = generator_criterion(fake_out, fake_img, real_img) + face_loss(fake_img, real_img)
        
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        optimizerG.step()

        # Loss for current batch before optimization 

        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        print("===> Epoch[{}]({}/{}): D(x) {:.4f} Loss_D: {:.4f}".format(epoch, iteration, 
                                                                         len(training_data_loader), 
                                                                         real_out.item(),
                                                                         d_loss.item()))

    batch_mse = ((fake_img - real_img) ** 2).data.mean()
    batch_ssim = ssim(fake_img, real_img).item()
    batch_psnr = 10 * log10(1 /batch_mse)  
    
    print('[{}/{}] Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f} PSNR {:.4f}'.format(
        epoch, num_epochs, 
        running_results['d_loss'] / running_results['batch_sizes'],
        running_results['g_loss'] / running_results['batch_sizes'],
        running_results['d_score'] / running_results['batch_sizes'],
        running_results['g_score'] / running_results['batch_sizes'], batch_psnr))
          
    # Save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(batch_psnr)
    results['ssim'].append(batch_ssim)

    if epoch % (num_epochs // 2) == 0:

        # Save model parameters
        torch.save(netG, out_model_path + 'SRGAN_FaceLoss_x%d_epoch_%d.pth' % (upscale_factor, epoch))
        #torch.save(netD, 'netD_x%d_epoch_%d.pt' % (upscale_factor, epoch))

        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'SRGAN_FaceLoss_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')

In [ ]:
num_epochs = 30

for epoch in range(1, num_epochs + 1):
    train(epoch)

===> Epoch[1](1/562): Discriminator Loss: 0.9993
===> Epoch[1](2/562): Discriminator Loss: 0.9970
===> Epoch[1](3/562): Discriminator Loss: 0.9943
===> Epoch[1](4/562): Discriminator Loss: 0.9913
===> Epoch[1](5/562): Discriminator Loss: 0.9868
===> Epoch[1](6/562): Discriminator Loss: 0.9821
===> Epoch[1](7/562): Discriminator Loss: 0.9775
===> Epoch[1](8/562): Discriminator Loss: 0.9638
===> Epoch[1](9/562): Discriminator Loss: 0.9658
===> Epoch[1](10/562): Discriminator Loss: 0.9838
===> Epoch[1](11/562): Discriminator Loss: 0.9580
===> Epoch[1](12/562): Discriminator Loss: 0.9389
===> Epoch[1](13/562): Discriminator Loss: 0.9404
===> Epoch[1](14/562): Discriminator Loss: 0.9213
===> Epoch[1](15/562): Discriminator Loss: 0.9184
===> Epoch[1](16/562): Discriminator Loss: 0.8913
===> Epoch[1](17/562): Discriminator Loss: 0.9425
===> Epoch[1](18/562): Discriminator Loss: 0.8845
===> Epoch[1](19/562): Discriminator Loss: 0.8804
===> Epoch[1](20/562): Discriminator Loss: 0.8703
===> Epoc

===> Epoch[1](164/562): Discriminator Loss: 0.2338
===> Epoch[1](165/562): Discriminator Loss: 0.2428
===> Epoch[1](166/562): Discriminator Loss: 0.2535
===> Epoch[1](167/562): Discriminator Loss: 0.1939
===> Epoch[1](168/562): Discriminator Loss: 0.2513
===> Epoch[1](169/562): Discriminator Loss: 0.1627
===> Epoch[1](170/562): Discriminator Loss: 0.1497
===> Epoch[1](171/562): Discriminator Loss: 0.2007
===> Epoch[1](172/562): Discriminator Loss: 0.1743
===> Epoch[1](173/562): Discriminator Loss: 0.3435
===> Epoch[1](174/562): Discriminator Loss: 0.1587
===> Epoch[1](175/562): Discriminator Loss: 0.2206
===> Epoch[1](176/562): Discriminator Loss: 0.1844
===> Epoch[1](177/562): Discriminator Loss: 0.2621
===> Epoch[1](178/562): Discriminator Loss: 0.2111
===> Epoch[1](179/562): Discriminator Loss: 0.2079
===> Epoch[1](180/562): Discriminator Loss: 0.1934
===> Epoch[1](181/562): Discriminator Loss: 0.1821
===> Epoch[1](182/562): Discriminator Loss: 0.2145
===> Epoch[1](183/562): Discrim

===> Epoch[1](325/562): Discriminator Loss: 0.2211
===> Epoch[1](326/562): Discriminator Loss: 0.2108
===> Epoch[1](327/562): Discriminator Loss: 0.2506
===> Epoch[1](328/562): Discriminator Loss: 0.2724
===> Epoch[1](329/562): Discriminator Loss: 0.2232
===> Epoch[1](330/562): Discriminator Loss: 0.1876
===> Epoch[1](331/562): Discriminator Loss: 0.2014
===> Epoch[1](332/562): Discriminator Loss: 0.2008
===> Epoch[1](333/562): Discriminator Loss: 0.2812
===> Epoch[1](334/562): Discriminator Loss: 0.2366
===> Epoch[1](335/562): Discriminator Loss: 0.1900
===> Epoch[1](336/562): Discriminator Loss: 0.2210
===> Epoch[1](337/562): Discriminator Loss: 0.2561
===> Epoch[1](338/562): Discriminator Loss: 0.2575
===> Epoch[1](339/562): Discriminator Loss: 0.1931
===> Epoch[1](340/562): Discriminator Loss: 0.1947
===> Epoch[1](341/562): Discriminator Loss: 0.2136
===> Epoch[1](342/562): Discriminator Loss: 0.2614
===> Epoch[1](343/562): Discriminator Loss: 0.3095
===> Epoch[1](344/562): Discrim

===> Epoch[1](486/562): Discriminator Loss: 0.1665
===> Epoch[1](487/562): Discriminator Loss: 0.1632
===> Epoch[1](488/562): Discriminator Loss: 0.1495
===> Epoch[1](489/562): Discriminator Loss: 0.1678
===> Epoch[1](490/562): Discriminator Loss: 0.1403
===> Epoch[1](491/562): Discriminator Loss: 0.1528
===> Epoch[1](492/562): Discriminator Loss: 0.1676
===> Epoch[1](493/562): Discriminator Loss: 0.1713
===> Epoch[1](494/562): Discriminator Loss: 0.1943
===> Epoch[1](495/562): Discriminator Loss: 0.2100
===> Epoch[1](496/562): Discriminator Loss: 0.1725
===> Epoch[1](497/562): Discriminator Loss: 0.1623
===> Epoch[1](498/562): Discriminator Loss: 0.2226
===> Epoch[1](499/562): Discriminator Loss: 0.1815
===> Epoch[1](500/562): Discriminator Loss: 0.1857
===> Epoch[1](501/562): Discriminator Loss: 0.1114
===> Epoch[1](502/562): Discriminator Loss: 0.2125
===> Epoch[1](503/562): Discriminator Loss: 0.1902
===> Epoch[1](504/562): Discriminator Loss: 0.1341
===> Epoch[1](505/562): Discrim

===> Epoch[2](85/562): Discriminator Loss: 0.1336
===> Epoch[2](86/562): Discriminator Loss: 0.1903
===> Epoch[2](87/562): Discriminator Loss: 0.1476
===> Epoch[2](88/562): Discriminator Loss: 0.1999
===> Epoch[2](89/562): Discriminator Loss: 0.1884
===> Epoch[2](90/562): Discriminator Loss: 0.2019
===> Epoch[2](91/562): Discriminator Loss: 0.1895
===> Epoch[2](92/562): Discriminator Loss: 0.1710
===> Epoch[2](93/562): Discriminator Loss: 0.1610
===> Epoch[2](94/562): Discriminator Loss: 0.2413
===> Epoch[2](95/562): Discriminator Loss: 0.1586
===> Epoch[2](96/562): Discriminator Loss: 0.1886
===> Epoch[2](97/562): Discriminator Loss: 0.2075
===> Epoch[2](98/562): Discriminator Loss: 0.1332
===> Epoch[2](99/562): Discriminator Loss: 0.1739
===> Epoch[2](100/562): Discriminator Loss: 0.1395
===> Epoch[2](101/562): Discriminator Loss: 0.2000
===> Epoch[2](102/562): Discriminator Loss: 0.1723
===> Epoch[2](103/562): Discriminator Loss: 0.1944
===> Epoch[2](104/562): Discriminator Loss: 0.

...

===> Epoch[7](499/562): Discriminator Loss: 0.0459
===> Epoch[7](500/562): Discriminator Loss: 0.0359
===> Epoch[7](501/562): Discriminator Loss: 0.0565
===> Epoch[7](502/562): Discriminator Loss: 0.0395
===> Epoch[7](503/562): Discriminator Loss: 0.0672
===> Epoch[7](504/562): Discriminator Loss: 0.0542
===> Epoch[7](505/562): Discriminator Loss: 0.0763
===> Epoch[7](506/562): Discriminator Loss: 0.0766
===> Epoch[7](507/562): Discriminator Loss: 0.0964
===> Epoch[7](508/562): Discriminator Loss: 0.0619
===> Epoch[7](509/562): Discriminator Loss: 0.0419
===> Epoch[7](510/562): Discriminator Loss: 0.0543
===> Epoch[7](511/562): Discriminator Loss: 0.0773
===> Epoch[7](512/562): Discriminator Loss: 0.0511
===> Epoch[7](513/562): Discriminator Loss: 0.0611
===> Epoch[7](514/562): Discriminator Loss: 0.0645
===> Epoch[7](515/562): Discriminator Loss: 0.0648
===> Epoch[7](516/562): Discriminator Loss: 0.0889
===> Epoch[7](517/562): Discriminator Loss: 0.0809
===> Epoch[7](518/562): Discrim

===> Epoch[8](99/562): Discriminator Loss: 0.0711
===> Epoch[8](100/562): Discriminator Loss: 0.0529
===> Epoch[8](101/562): Discriminator Loss: 0.0425
===> Epoch[8](102/562): Discriminator Loss: 0.0494
===> Epoch[8](103/562): Discriminator Loss: 0.0401
===> Epoch[8](104/562): Discriminator Loss: 0.0757
===> Epoch[8](105/562): Discriminator Loss: 0.0340
===> Epoch[8](106/562): Discriminator Loss: 0.0659
===> Epoch[8](107/562): Discriminator Loss: 0.0493
===> Epoch[8](108/562): Discriminator Loss: 0.0714
===> Epoch[8](109/562): Discriminator Loss: 0.0668
===> Epoch[8](110/562): Discriminator Loss: 0.0513
===> Epoch[8](111/562): Discriminator Loss: 0.0383
===> Epoch[8](112/562): Discriminator Loss: 0.0416
===> Epoch[8](113/562): Discriminator Loss: 0.0556
===> Epoch[8](114/562): Discriminator Loss: 0.0515
===> Epoch[8](115/562): Discriminator Loss: 0.0639
===> Epoch[8](116/562): Discriminator Loss: 0.0816
===> Epoch[8](117/562): Discriminator Loss: 0.0460
===> Epoch[8](118/562): Discrimi

===> Epoch[8](260/562): Discriminator Loss: 0.0568
===> Epoch[8](261/562): Discriminator Loss: 0.0377
===> Epoch[8](262/562): Discriminator Loss: 0.0356
===> Epoch[8](263/562): Discriminator Loss: 0.0309
===> Epoch[8](264/562): Discriminator Loss: 0.0225
===> Epoch[8](265/562): Discriminator Loss: 0.0531
===> Epoch[8](266/562): Discriminator Loss: 0.0179
===> Epoch[8](267/562): Discriminator Loss: 0.0400
===> Epoch[8](268/562): Discriminator Loss: 0.0246
===> Epoch[8](269/562): Discriminator Loss: 0.0349
===> Epoch[8](270/562): Discriminator Loss: 0.0503
===> Epoch[8](271/562): Discriminator Loss: 0.0282
===> Epoch[8](272/562): Discriminator Loss: 0.0179
===> Epoch[8](273/562): Discriminator Loss: 0.0303
===> Epoch[8](274/562): Discriminator Loss: 0.0369
===> Epoch[8](275/562): Discriminator Loss: 0.0207
===> Epoch[8](276/562): Discriminator Loss: 0.0202
===> Epoch[8](277/562): Discriminator Loss: 0.0187
===> Epoch[8](278/562): Discriminator Loss: 0.0748
===> Epoch[8](279/562): Discrim

===> Epoch[8](421/562): Discriminator Loss: 0.0248
===> Epoch[8](422/562): Discriminator Loss: 0.0236
===> Epoch[8](423/562): Discriminator Loss: 0.0219
===> Epoch[8](424/562): Discriminator Loss: 0.0524
===> Epoch[8](425/562): Discriminator Loss: 0.0234
===> Epoch[8](426/562): Discriminator Loss: 0.0194
===> Epoch[8](427/562): Discriminator Loss: 0.0235
===> Epoch[8](428/562): Discriminator Loss: 0.0557
===> Epoch[8](429/562): Discriminator Loss: 0.0359
===> Epoch[8](430/562): Discriminator Loss: 0.0674
===> Epoch[8](431/562): Discriminator Loss: 0.0425
===> Epoch[8](432/562): Discriminator Loss: 0.0864
===> Epoch[8](433/562): Discriminator Loss: 0.0218
===> Epoch[8](434/562): Discriminator Loss: 0.0323
===> Epoch[8](435/562): Discriminator Loss: 0.0380
===> Epoch[8](436/562): Discriminator Loss: 0.0663
===> Epoch[8](437/562): Discriminator Loss: 0.0580
===> Epoch[8](438/562): Discriminator Loss: 0.0368
===> Epoch[8](439/562): Discriminator Loss: 0.0313
===> Epoch[8](440/562): Discrim

===> Epoch[9](19/562): Discriminator Loss: 0.0336
===> Epoch[9](20/562): Discriminator Loss: 0.0284
===> Epoch[9](21/562): Discriminator Loss: 0.0176
===> Epoch[9](22/562): Discriminator Loss: 0.0596
===> Epoch[9](23/562): Discriminator Loss: 0.0387
===> Epoch[9](24/562): Discriminator Loss: 0.0393
===> Epoch[9](25/562): Discriminator Loss: 0.0389
===> Epoch[9](26/562): Discriminator Loss: 0.0186
===> Epoch[9](27/562): Discriminator Loss: 0.0315
===> Epoch[9](28/562): Discriminator Loss: 0.0398
===> Epoch[9](29/562): Discriminator Loss: 0.0326
===> Epoch[9](30/562): Discriminator Loss: 0.0286
===> Epoch[9](31/562): Discriminator Loss: 0.0431
===> Epoch[9](32/562): Discriminator Loss: 0.0255
===> Epoch[9](33/562): Discriminator Loss: 0.0304
===> Epoch[9](34/562): Discriminator Loss: 0.0212
===> Epoch[9](35/562): Discriminator Loss: 0.0210
===> Epoch[9](36/562): Discriminator Loss: 0.0388
===> Epoch[9](37/562): Discriminator Loss: 0.0539
===> Epoch[9](38/562): Discriminator Loss: 0.0258
